In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler


from vgg16_2 import VGG_16_2


from data_loader_withid_single_label_hair import get_loader
from data_loader_withid_single_label_hair import get_dataset

import math

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0'
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair']
celeba_crop_size = 224
image_size = 224 
batch_size = 30
num_workers = 0
mode1 = 'train' 
mode2 = 'test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #"cuda:0" if torch.cuda.is_available() else 
torch.set_default_tensor_type('torch.cuda.FloatTensor')


 


In [3]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [4]:
celeba_loader_test2, data_test2 = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, 1,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...


In [5]:
print(len(data_train))
print(len(data_test))

87293
9650


# Validate

In [6]:
def validate_single_label(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)


In [7]:
def validate_single_label_stoch(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



# Noisy Network

In [8]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        self.rhos.requires_grad = True
        self.locs.requires_grad = True
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)*mask
        return  self.locs + self.scales()*epsilon
                                 
                               
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise



class vgg_syn(nn.Module):

    def __init__(self, model_features, model_classifier, min_scale,max_scale, given_locs, given_scale):
        super(vgg_syn, self).__init__()
        

        self.intermed = NoisyActivation( given_locs, given_scale, min_scale, max_scale)
        self.model_pt2 =  torch.nn.Sequential(*(list(model_features)))
        self.model_pt3 = model_classifier
        #self.components = components
        for child in itertools.chain(self.model_pt2, self.model_pt3): #self.model_pt2 #(self.model_pt2, 
            for param in child.parameters():
                param.requires_grad = False
            if isinstance(child, nn.modules.batchnorm._BatchNorm):
                child.eval()
                child.affine = False
                child.track_running_stats = False
                
        self.intermed.rhos.reuires_grad = True
        self.intermed.locs.reuires_grad = True
                                 
    def forward(self, img, mask):
                                 
        img = img
        x = self.intermed(img, mask)
        noisy = x.detach()
       
        x = self.model_pt2(x)                    
        x = x.view(img.size(0), -1)
        x = self.model_pt3(x)                                 

        return x, noisy
    
                                 
 

# Test adv

In [9]:
def test_adversary(model, model_syn):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        _, noisy = model_syn(im)
        preds = model(noisy)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        del im
        del preds

        del values

        #print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))

    print(size)
    print(correct)
    return size,correct





In [10]:
def validate_mask(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        im = im*mask
        preds = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)



In [11]:
def validate_per_example_mask(model, mult_mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        im_2 = im*mult_mask
        preds = model(im_2)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        #del im_2
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



#  Train   noisy adversary

In [12]:
def train_adversary(model, model_syn, criterion, optimizer):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    model_syn.eval()
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        print("iteration no", i)
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        
        
        
        optimizer.zero_grad()
        _, noisy = model_syn(im)
        output = model(noisy)
        

        loss = criterion(output, labels)
        
        del im
        del noisy
        del _
        
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()

        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
        #print(list(filter(lambda p: p.requires_grad, model.parameters()))[0][0])
        







In [13]:
def train_adversary_mask(model, model_syn, criterion, optimizer, mask):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    model_syn.eval()
    for i, (im, labels) in enumerate(celeba_loader_train):
  
        print("iteration no", i)
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        
        
        
        optimizer.zero_grad()
        _, noisy = model_syn(im, mask)
        output = model(noisy)
        
        


        loss = criterion(output, labels)
        
        del im
        del noisy
        del _
        
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()

        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
        #print(list(filter(lambda p: p.requires_grad, model.parameters()))[0][0])
        








# experiments

In [14]:

model_hair = VGG_16_2()

model_hair.load_state_dict(torch.load("celeba-black-hair-2-class-84l4",  map_location=device))

<All keys matched successfully>

In [15]:
model = VGG_16_2()


criterion = nn.NLLLoss()
mus = torch.zeros((3,224,224))
scale = torch.ones((3,224,224))*0.001
model_syn = vgg_syn(model.convnet, model.fc ,0, 5 ,mus, scale )
model_syn.load_state_dict(torch.load("celeba-smiling-avg-cloak", map_location=device))

<All keys matched successfully>

In [16]:
criterion = nn.CrossEntropyLoss().to(device)

In [17]:
for child in itertools.chain(model_hair.fc, model_hair.convnet): #self.model_pt2 #(self.model_pt2, 
    for param in child.parameters():
        param.requires_grad = True
print(model_hair.fc[-1])


Linear(in_features=4096, out_features=2, bias=True)


In [18]:
d = list(filter(lambda p: p.requires_grad, model_hair.parameters()))
print(len(d))

32


In [19]:
mult_mask = torch.where(model_syn.intermed.scales()>4.5, torch.zeros(model_syn.intermed.scales().shape), torch.ones(model_syn.intermed.scales().shape))

In [20]:
lr =  0.0001  
for epoch in range (1): # real
   
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_hair.parameters()), lr=lr, weight_decay=0)
    cor, size = train_adversary_mask(model_hair,model_syn, criterion, optimizer, mult_mask)
    print (cor,size)
 
    test_adversary_mask(model_hair, mult_mask)
    
    #wd helps
    if (epoch>  5):
        lr  = 0.00001
        print(lr)

iteration no 0
tensor(0.9223, grad_fn=<NllLossBackward>) loss
0.5333333333333333
iteration no 1
tensor(0.6953, grad_fn=<NllLossBackward>) loss
0.5166666666666667
iteration no 2
tensor(1.1526, grad_fn=<NllLossBackward>) loss
0.5111111111111111
iteration no 3
tensor(0.6658, grad_fn=<NllLossBackward>) loss
0.5416666666666666
iteration no 4
tensor(0.6889, grad_fn=<NllLossBackward>) loss
0.5466666666666666
iteration no 5
tensor(0.6768, grad_fn=<NllLossBackward>) loss
0.5555555555555556
iteration no 6
tensor(0.6848, grad_fn=<NllLossBackward>) loss
0.5571428571428572
iteration no 7
tensor(0.7169, grad_fn=<NllLossBackward>) loss
0.5458333333333333
iteration no 8
tensor(0.7029, grad_fn=<NllLossBackward>) loss
0.5407407407407407
iteration no 9
tensor(0.7386, grad_fn=<NllLossBackward>) loss
0.52
iteration no 10
tensor(0.6814, grad_fn=<NllLossBackward>) loss
0.5272727272727272
iteration no 11
tensor(0.7049, grad_fn=<NllLossBackward>) loss
0.5138888888888888
iteration no 12
tensor(0.6933, grad_fn=<

KeyboardInterrupt: 